In [31]:
import pandas as pd

# Define a function to parse the GEDCOM file
def parse_gedcom(file_path):
    individuals = {}
    current_individual = None
    current_tag = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.startswith('0 @I'):
                current_individual = line.split('@')[2]
                individuals[current_individual] = {}
            elif line.startswith('1'):
                current_tag = line.split(' ')[1]
                if current_individual is not None:
                    individuals[current_individual][current_tag] = []
            elif line.startswith('2'):
                value = ' '.join(line.split(' ')[2:])
                if current_individual is not None and current_tag is not None:
                    individuals[current_individual][current_tag].append(value)

    return individuals

# Parse the GEDCOM file
individuals = parse_gedcom('BothHarmonSniderTree.ged')

# Create a DataFrame for individuals
individual_data = []
for individual_id, individual in individuals.items():
    data = {'ID': individual_id}
    for tag, values in individual.items():
        if tag == 'NAME':
            data[tag] = ' '.join(values)
        else:
            data[tag] = values
    individual_data.append(data)

individual_df = pd.DataFrame(individual_data)

# Print the DataFrame
print(individual_df)


      ID           NAME SEX _UID _FSFTID           CHAN  \
0   INDI  Lina Humiston  []   []      []  [21 JUL 2024]   

                                                BIRT  \
0  [29 JUL 1797, Colesville, Broome, New York, Un...   

                                                DEAT  \
0  [25 MAY 1869, Richfield Center, Lucas, Ohio, U...   

                                                BURI                  BAPL  \
0  [28 MAY 1869, Sylvania, Lucas, Ohio, United St...  [19 MAR 2002, ARIZO]   

                   CONL                  WAC                  ENDL  \
0  [19 MAR 2002, ARIZO]  [9 MAY 2003, ARIZO]  [21 AUG 2004, MEXIC]   

                   SLGC FAMS  
0  [22 JAN 2005, MEXIC]   []  


In [33]:
import pandas as pd

# Define a function to parse the GEDCOM file
def parse_gedcom(file_path):
    individuals = {}
    current_individual = None
    current_tag = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.startswith('0 @I'):
                current_individual = line.split('@')[2]
                individuals[current_individual] = {}
            elif line.startswith('1'):
                current_tag = line.split(' ')[1]
                if current_individual is not None:
                    individuals[current_individual][current_tag] = []
            elif line.startswith('2'):
                value = ' '.join(line.split(' ')[2:])
                if current_individual is not None and current_tag is not None:
                    individuals[current_individual][current_tag].append(value)

    return individuals


# Parse the GEDCOM file
individuals = parse_gedcom('BothHarmonSniderTree.ged')

# Create a DataFrame for individuals
individual_data = []
for individual_id, individual in individuals.items():
    data = {'ID': individual_id}
    for tag, values in individual.items():
        if tag == 'NAME':
            data[tag] = ' '.join(values)
        else:
            data[tag] = values
    individual_data.append(data)

individual_df = pd.DataFrame(individual_data)

# Print the DataFrame
print(individual_df)


      ID           NAME SEX _UID _FSFTID           CHAN  \
0   INDI  Lina Humiston  []   []      []  [21 JUL 2024]   

                                                BIRT  \
0  [29 JUL 1797, Colesville, Broome, New York, Un...   

                                                DEAT  \
0  [25 MAY 1869, Richfield Center, Lucas, Ohio, U...   

                                                BURI                  BAPL  \
0  [28 MAY 1869, Sylvania, Lucas, Ohio, United St...  [19 MAR 2002, ARIZO]   

                   CONL                  WAC                  ENDL  \
0  [19 MAR 2002, ARIZO]  [9 MAY 2003, ARIZO]  [21 AUG 2004, MEXIC]   

                   SLGC FAMS  
0  [22 JAN 2005, MEXIC]   []  
